# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_file = "../WeatherPy/city_data.csv"
city_df = pd.read_csv(city_file)
city_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Okhotsk,59.38,143.30,31.19,96,99,14.05,RU,1603306297
1,1,Edmundston,47.37,-68.33,39.99,93,90,13.87,CA,1603306529
2,2,Campo de Criptana,39.40,-3.12,55.99,76,100,4.00,ES,1603306530
3,3,Ushuaia,-54.80,-68.30,55.40,43,75,16.11,AR,1603306261
4,4,Talnakh,69.49,88.40,12.20,85,100,13.42,RU,1603306530


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations=city_df[["Lat","Lng"]]
humidity=city_df["Humidity"]
gmaps.configure(api_key=g_key)

In [4]:
figure_layout = {
    'width': '100%',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_narrow=city_df[(city_df["Humidity"]<70) & (city_df["Max Temp"]<75)& (city_df["Max Temp"]>65) & (city_df["Cloudiness"]<10)]
print(len(city_narrow))
city_narrow.head()

22


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,Darnah,32.77,22.64,68.77,68,4,13.49,LY,1603306541
63,63,Ocampo,21.65,-101.50,70.97,30,0,9.04,MX,1603306542
111,111,Rikitea,-23.12,-134.97,73.87,66,5,8.52,PF,1603305720
130,130,Pacific Grove,36.62,-121.92,71.60,68,1,4.70,US,1603306555
202,202,Vredendal,-31.67,18.50,70.90,29,0,12.24,ZA,1603306569


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=city_narrow[["City","Lat","Lng","Country"]]
hotel_df["Hotel"]=""
hotel_df.dropna()
hotel_df=hotel_df.reset_index()


C:\Users\ajhib\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key }

for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    query=requests.get(base_url,params=params)
    query=query.json()
    try:
        hotel_df.loc[index,"Hotel"]=query["results"][0]["name"]
    except:
        print(f"Missing {index}")
    
       
    

Missing 20


In [8]:
hotelf_df=hotel_df.copy()
hotelf_df.drop(['index'], axis=1)

,City,Lat,Lng,Country,Hotel
0,Darnah,32.77,22.64,LY,زاوية الرفاعي
1,Ocampo,21.65,-101.50,MX,Hotel Estancia del Vaquero
2,Rikitea,-23.12,-134.97,PF,Pension Maro'i
3,Pacific Grove,36.62,-121.92,US,Lovers Point Inn
4,Vredendal,-31.67,18.50,ZA,Tharrakamma Guest House
5,Opuwo,-18.06,13.84,NaN,ABBA Guesthouse
6,Cedar City,37.68,-113.06,US,Hampton Inn Cedar City
7,Portales,34.19,-103.33,US,Holiday Inn Express & Suites Portales
8,Karoi,-16.81,29.69,ZW,Bliss Lodge Karoi
9,Neuquén,-38.95,-68.06,AR,Hotel del Comahue


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelf_df.iterrows()]
locations = hotelf_df[["Lat", "Lng"]]
info=hotel_df[["Hotel","City","Country"]]

In [10]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…